**<h1>SEO Analytics Project**

<h3>Các bước thực hiện:

1. Đọc dữ liệu từ cơ sở dữ liệu.
2. Xử lý dữ liệu date đúng định dạng
3. Loại bỏ các cột không cần thiết.
4. Chuyển các dữ liệu lệch cột về đúng vị trí.
5. Tách tên miền từ cột MA Referrer.
6. Tìm vị trí người dùng thông qua IP Address.
7. Phân loại các nội dung tìm kiếm.
8. Phân loại người dùng qua Email.
9. Lọc các lượt truy cập 1 lần không tương tác (Bounce Rate).

In [212]:
# Gọi các thư viện cần thiết
import pandas as pd
import tldextract # tldextract giúp lấy tên miền
import geoip2.database # geoip2 giúp lấy thông tin địa lý từ địa chỉ IP
from tqdm import tqdm # hiển thị tiến trình
from sqlalchemy import create_engine # SQLAlchemy giúp kết nối và tương tác với cơ sở dữ liệu
import urllib.parse 

<h2>Đọc dữ liệu

In [ ]:
server = 'longth.database.windows.net'
database = 'longth'
username = 'student'
password = 'Rm!t2002'

driver = 'ODBC Driver 17 for SQL Server' 

params = urllib.parse.quote_plus(
    f"DRIVER={driver};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    "Encrypt=yes;"
    "TrustServerCertificate=no;"
    "Connection Timeout=30;"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

In [ ]:
def sql_to_df(table):
    query = f'Select * from dbo.{table}'
    df = pd.read_sql(query, engine)
    return df

In [ ]:
# Danh sách tên bảng
tables = ['Uniace_1', 'Uniace_2', 'Uniace_3']

dataframes = {}

for table in tables:
    dataframes[table] = sql_to_df(table)

# Truy cập từng bảng:
df1 = dataframes['Uniace_1']
df2 = dataframes['Uniace_2']
df3 = dataframes['Uniace_3']

<h2>Định dạng cột Date

In [ ]:
def clean_date(dataframe):
    dataframe = dataframe.str.replace('/','-')

    # Tạo cột 'Date_1' và 'Hour_1' từ cột 'Date' để tách ngày và giờ
    dataframe['Date_1'] = dataframe['Date'].apply(lambda x: str(x).split(' ')[0] if pd.notnull(x) else None)  # Tách ngày từ cột 'Date'
    dataframe['Hour_1'] = dataframe['Date'].apply(lambda x: str(x).split(' ')[1] if pd.notnull(x) and len(str(x).split(' ')) > 1 else None)  # Tách giờ từ cột 'Date'

    # Chuyển đổi cột 'Date_1' sang định dạng datetime và định dạng lại thành 'YYYY-MM-DD' để Power BI có thể nhận diện
    dataframe['Date_1'] = pd.to_datetime(dataframe['Date_1'], dayfirst=True, errors='coerce')  # Chuyển đổi cột 'Date_1' sang định dạng datetime
    dataframe['Date_1'] = dataframe['Date_1'].dt.strftime('%Y-%m-%d')
    
    return dataframe




In [ ]:
df1 = clean_date(df1)
df2 = clean_date(df2)
df3 = clean_date(df3)

<h2>Loại bỏ các cột không cần thiết

In [214]:
df1.drop(columns=['List Id','Form Id','Campaign Id','Campaign Name','Scenario Id','URL','Link'
], inplace=True)
df2.drop(columns=['List Id','Form Id','Campaign Id','Campaign Name','Scenario Id','URL','Link'
], inplace=True)
df3.drop(columns=['List Id','Form Id','Campaign Id','Campaign Name','Scenario Id','URL','Link'
], inplace=True)

**<h2>Lọc các dòng có dữ liệu bị lệch của bảng 3 sau đó đưa về đúng vị trí.**

In [215]:
# Sử dụng regex để lọc ra các IP Address bắt đầu bằng số 1-9 vào df3_f và tách các IP Address không bắt đầu bằng số 1-9 vào df3_s
df3_f = df3[df3['IP Address'].str.contains(r'^[1-9]',regex=True,na=True) == True]
df3_s = df3[df3['IP Address'].str.contains(r'^[1-9]',regex=True,na=True) == False]

In [216]:
df3_s

,Email,Type,Name,Title,MA URL,MA Referrer,ma_path,IP Address,cuid,Date,Message Id,Template Id,Tag
4340,NaN,page,Chủ nghĩa khắc kỷ là gì? Cách ứng dụng vào cuộ...,Chủ nghĩa khắc kỷ là gì? Cách ứng dụng vào cuộ...,https://uniace.vn/chu-nghia-khac-ky-la-gi/,https://r.search.yahoo.com/_ylt=AwrwBpVlnx1hq6...,_ylu=Y29sbwMEcG9zAzQEdnRpZAMEc2VjA3Ny/RV=2/RE=...,/chu-nghia-khac-ky-la-gi/,115.76.26.140,f025d187-9ec2-4571-930f-38b2ecdddff7,NaN,NaN,NaN
6200,NaN,page,Áp lực cuộc sống là do đâu? 4 cách vượt qua áp...,Áp lực cuộc sống là do đâu? 4 cách vượt qua áp...,https://uniace.vn/ap-luc-cuoc-song-la-do-dau/,https://r.search.yahoo.com/_ylt=Awr9Jm8zxRthUQ...,_ylu=Y29sbwNncTEEcG9zAzEEdnRpZAMEc2VjA3Ny/RV=2...,/ap-luc-cuoc-song-la-do-dau/,113.175.10.210,4196ce81-3e4d-4892-a2c5-dd153ed7c793,NaN,NaN,NaN
7475,NaN,page,Cách lấy dữ liệu từ bảng này sang bảng khác tr...,Cách lấy dữ liệu từ bảng này sang bảng khác tr...,https://uniace.vn/cach-lay-du-lieu-tu-bang-nay...,https://r.search.yahoo.com/_ylt=Awr4xJQ2HRth7f...,_ylu=Y29sbwNncTEEcG9zAzcEdnRpZAMEc2VjA3Ny/RV=2...,/cach-lay-du-lieu-tu-bang-nay-sang-bang-khac-t...,118.71.69.164,1b33d6e0-c8da-4827-9c85-d8cbb4a69446,NaN,NaN,NaN
10549,NaN,page,Hướng dẫn Power Query cơ bản cho người mới bắt...,Hướng dẫn Power Query cơ bản cho người mới bắt...,https://uniace.vn/huong-dan-power-query-co-ban...,https://r.search.yahoo.com/_ylt=Awr4xJMOExlhGd...,_ylu=Y29sbwNncTEEcG9zAzQEdnRpZAMEc2VjA3Ny/RV=2...,/huong-dan-power-query-co-ban-cho-nguoi-moi-ba...,116.102.41.29,41da7282-162d-4a00-be83-98dcff553817,NaN,NaN,NaN
10681,NaN,page,Hướng dẫn sử dụng Power Query để chuẩn hóa dữ ...,Hướng dẫn sử dụng Power Query để chuẩn hóa dữ ...,https://uniace.vn/huong-dan-su-dung-power-quer...,https://r.search.yahoo.com/_ylt=Awr9DtI8DRlhw8...,_ylu=Y29sbwNncTEEcG9zAzYEdnRpZAMEc2VjA3Ny/RV=2...,/huong-dan-su-dung-power-query-de-chuan-hoa-du...,116.102.41.29,41da7282-162d-4a00-be83-98dcff553817,NaN,NaN,NaN
12553,NaN,page,Hướng Dẫn Cách Copy Giữ Nguyên Giá Trị Trong E...,Hướng Dẫn Cách Copy Giữ Nguyên Giá Trị Trong E...,https://uniace.vn/cach-copy-giu-nguyen-gia-tri...,https://r.search.yahoo.com/_ylt=AwrwZeUa1xdhyz...,_ylu=Y29sbwNzZzMEcG9zAzMEdnRpZAMEc2VjA3Ny/RV=2...,/cach-copy-giu-nguyen-gia-tri-trong-excel/,14.175.156.166,02ba7bad-4569-4207-9a93-202fc4b9a792,NaN,NaN,NaN
12631,NaN,page,Hướng Dẫn Cách Copy Giữ Nguyên Giá Trị Trong E...,Hướng Dẫn Cách Copy Giữ Nguyên Giá Trị Trong E...,https://uniace.vn/cach-copy-giu-nguyen-gia-tri...,https://r.search.yahoo.com/_ylt=AwrS_Alzxhdhsx...,_ylu=Y29sbwNncTEEcG9zAzIEdnRpZAMEc2VjA3Ny/RV=2...,/cach-copy-giu-nguyen-gia-tri-trong-excel/,113.160.214.65,db772472-a05f-440b-80d6-ccfdc4419937,NaN,NaN,NaN
16707,NaN,page,Hướng Dẫn Cách Kết Nối Đến Power Query,Hướng Dẫn Cách Kết Nối Đến Power Query,https://uniace.vn/huong-dan-power-query-co-ban...,https://r.search.yahoo.com/_ylt=Awr9Dux5bhZhP9...,_ylu=Y29sbwNncTEEcG9zAzEwBHZ0aWQDBHNlYwNzcg--/...,/huong-dan-power-query-co-ban-cho-nguoi-moi-ba...,116.102.41.29,41da7282-162d-4a00-be83-98dcff553817,NaN,NaN,NaN
16716,NaN,page,Power Pivot Là Gì? Tìm Hiểu Ứng Dụng Của Power...,Power Pivot Là Gì? Tìm Hiểu Ứng Dụng Của Power...,https://uniace.vn/power-pivot-la-gi/,https://r.search.yahoo.com/_ylt=Awr9DuQOPhZhNA...,_ylu=Y29sbwNncTEEcG9zAzQEdnRpZAMEc2VjA3Ny/RV=2...,/power-pivot-la-gi/,116.102.41.29,41da7282-162d-4a00-be83-98dcff553817,NaN,NaN,NaN
19334,NaN,page,Hướng Dẫn Cách Copy Giữ Nguyên Giá Trị Trong E...,Hướng Dẫn Cách Copy Giữ Nguyên Giá Trị Trong E...,https://uniace.vn/cach-copy-giu-nguyen-gia-tri...,https://r.search.yahoo.com/_ylt=AwrPg08HkxRhED...,_ylu=Y29sbwNzZzMEcG9zAzEEdnRpZAMEc2VjA3Ny/RV=2...,/cach-copy-giu-nguyen-gia-tri-trong-excel/,103.156.42.94,a4372989-177e-43c1-b07a-c7e9fa312d33,NaN,NaN,NaN


In [217]:
# Đưa các dữ liệu bj lệch về đúng vị trí
df3_s.loc[:,'IP Address'] = df3_s.loc[:,'cuid']
df3_s.loc[:,'cuid'] = df3_s.loc[:,'Date']
df3_s.loc[:,'Date'] = df3_s.loc[:,'Message Id']

In [218]:
df3_s.head(3)

,Email,Type,Name,Title,MA URL,MA Referrer,ma_path,IP Address,cuid,Date,Message Id,Template Id,Tag
4340,NaN,page,Chủ nghĩa khắc kỷ là gì? Cách ứng dụng vào cuộ...,Chủ nghĩa khắc kỷ là gì? Cách ứng dụng vào cuộ...,https://uniace.vn/chu-nghia-khac-ky-la-gi/,https://r.search.yahoo.com/_ylt=AwrwBpVlnx1hq6...,_ylu=Y29sbwMEcG9zAzQEdnRpZAMEc2VjA3Ny/RV=2/RE=...,115.76.26.140,f025d187-9ec2-4571-930f-38b2ecdddff7,NaN,NaN,NaN,NaN
6200,NaN,page,Áp lực cuộc sống là do đâu? 4 cách vượt qua áp...,Áp lực cuộc sống là do đâu? 4 cách vượt qua áp...,https://uniace.vn/ap-luc-cuoc-song-la-do-dau/,https://r.search.yahoo.com/_ylt=Awr9Jm8zxRthUQ...,_ylu=Y29sbwNncTEEcG9zAzEEdnRpZAMEc2VjA3Ny/RV=2...,113.175.10.210,4196ce81-3e4d-4892-a2c5-dd153ed7c793,NaN,NaN,NaN,NaN
7475,NaN,page,Cách lấy dữ liệu từ bảng này sang bảng khác tr...,Cách lấy dữ liệu từ bảng này sang bảng khác tr...,https://uniace.vn/cach-lay-du-lieu-tu-bang-nay...,https://r.search.yahoo.com/_ylt=Awr4xJQ2HRth7f...,_ylu=Y29sbwNncTEEcG9zAzcEdnRpZAMEc2VjA3Ny/RV=2...,118.71.69.164,1b33d6e0-c8da-4827-9c85-d8cbb4a69446,NaN,NaN,NaN,NaN


In [219]:
# Gộp các dữ liệu df3_f và df3_s lại với nhau sau khi đã đưa về đúng vị trí
df3_f = pd.concat([df3_f,df3_s],axis=0)

In [220]:
df3_f.head(3)

,Email,Type,Name,Title,MA URL,MA Referrer,ma_path,IP Address,cuid,Date,Message Id,Template Id,Tag
0,lieudung27072000@gmail.com,page,HIGH PERFORMER - uniace.vn,HIGH PERFORMER - uniace.vn,https://uniace.vn/uhp/,https://www.google.com/,/uhp/,123.20.35.106,064b39d4-db8c-4152-8e0c-570390f6672a,21-08-2021 23:59:55,449.0,NaN,NaN
1,NaN,page,MASTERING DATA ANALYTICS - uniace.vn,MASTERING DATA ANALYTICS - uniace.vn,https://uniace.vn/mda,NaN,/mda,171.251.234.135,ba5cdde2-1612-41f8-9b77-1f5421f81634,21-08-2021 23:58:54,29.0,NaN,NaN
2,NaN,page,Chương Trình Young Talent Program - Uniace Việ...,Chương Trình Young Talent Program - Uniace Việ...,https://uniace.vn/vyt,NaN,/vyt,125.235.231.17,fadcb9fd-e30a-41d3-87bd-c254b0dff525,21-08-2021 23:58:28,276.0,NaN,NaN


<h3>Xử lý các dữ liệu lệch tương tự với bảng df1 và df2

In [221]:
df2_f = df2[df2['IP Address'].str.contains(r'^[1-9]',regex=True,na=True) == True]
df2_s = df2[df2['IP Address'].str.contains(r'^[1-9]',regex=True,na=True) == False]

In [222]:
# Bảng df2 có thêm cột 'MA Referrer' bị lệch vị trí nên ta tạo điều kiện lọc
bad_utms = [
    'utm_medium=1&amp',
    'utm_medium=4&amp',
    'utm_medium=6&amp',
    'utm_medium=9&amp',
]
cond = df2_s['MA Referrer'].isin(bad_utms)

In [223]:
cond.value_counts()

MA Referrer
True     176
False     13
Name: count, dtype: int64

In [224]:
df2_s.loc[cond,'MA Referrer'] = df2_s.loc[cond,'IP Address']
df2_s.loc[:,'IP Address'] = df2_s.loc[:,'Date']
df2_s.loc[:,'cuid'] = df2_s.loc[:,'Message Id']
df2_s.loc[:,'Date'] = df2_s.loc[:,'Template Id']

C:\Users\Dell Mobile 868\AppData\Local\Temp\ipykernel_416\3733288775.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_s.loc[cond,'MA Referrer'] = df2_s.loc[cond,'IP Address']


In [225]:
df2_f = pd.concat([df2_f,df2_s],axis=0)

In [226]:
df1_f = df1[df1['IP Address'].str.contains(r'^[1-9]',regex=True,na=True) == True]
df1_s = df1[df1['IP Address'].str.contains(r'^[1-9]',regex=True,na=True) == False]
print('Số dòng lệch giá trị: ',len(df1_s))

Số dòng lệch giá trị:  0


df1_s không có giá giá trị bị lệch nên không cần xử lý.

In [227]:
df1_f = pd.concat([df1_f,df1_s],axis=0)

<h3>Gộp bảng hoàn chỉnh

In [228]:
df = pd.concat([df1_f,df2_f,df3_f],axis=0)
df.reset_index(drop=True, inplace=True)

<h3>Sử dụng regex để tách tên miền

In [229]:
df['Refer'] = df['MA Referrer'].str.extract(r'^(?:https?://|android-app://)?(?:www.)?([^/]+)') # Lấy tên miền từ cột MA Referrer


In [230]:
df['Refer'].unique()

array(['google.com', 'google.com.vn', 'coccoc.com', nan, 'uniace.vn',
       'chgifgh.r.af.d.sendibt2.com', 'google.co.kr', 'google.com.au',
       'chgifgh.r.bh.d.sendibt3.com', 'bing.com', 'beacons.ai',
       'vn.search.yahoo.com', 'google.co.uk', 'submit.jotform.com',
       'form.jotform.com', 'm.facebook.com', 'facebook.com', 'angkoo.com',
       'google.com.sg', 'google.co.jp', 'beacons.page',
       'com.google.android.googlequicksearchbox', 'google.se',
       'l.facebook.com', 'google.com.ph', 'instabio.cc', 'id.zalo.me',
       'search.yahoo.com', 'baidu.com', 'discovery.koc.com.vn',
       'org.telegram.messenger', 'sg.search.yahoo.com', 'mlr.itim.vn',
       'com.google.android.gm', 'google.de', 'google.com.hk', 'google.ca',
       'chat.zalo.me', 'laban.vn', 'duckduckgo.com', 'google.fr',
       'jp.zaloapp.com', 'yandex.ru', 'ecosia.org', 'player.vimeo.com',
       'l.messenger.com', 'l.instagram.com',
       'uniace-operation.atlassian.net', 'google.com.tw',
       'r.s

In [231]:
df['Refer'].value_counts().head()

Refer
uniace.vn        30053
google.com       16474
google.com.vn     4124
beacons.page      2599
coccoc.com        1633
Name: count, dtype: int64

<h3>Sử dụng thư viện tlextract để tách các tên miền chính

In [232]:
def get_main_domain(url):
    if pd.isna(url) or url in bad_utms:
        return None
    ext = tldextract.extract(url)
    return ext.domain  # chỉ lấy phần 'google' trong 'mail.google.com'

# Áp dụng vào cột 'Refer'
df['Main_Domain'] = df['Refer'].apply(get_main_domain)

In [233]:
df['Main_Domain'].value_counts().head()

Main_Domain
uniace      30053
google      20750
beacons      2754
facebook     2316
coccoc       1633
Name: count, dtype: int64

<h2>Sử dụng thư viện geoip2 để tìm vị trí thành phố và quốc gia của người dùng thông qua địa chỉ IP.

In [234]:
# Đọc cơ sở dữ liệu GeoLite2
reader = geoip2.database.Reader("D:\Download\VVA\download\GeoLite2-City_20250516\GeoLite2-City.mmdb")

# Hàm tìm thông tin từ IP
def get_location(ip):
    try:
        response = reader.city(ip)
        country = response.country.name
        city = response.city.name
        return pd.Series([country, city])
    except:
        return pd.Series([None, None])

In [235]:
# Áp dụng lên toàn bộ DataFrame (dùng tqdm để hiển thị tiến độ)
tqdm.pandas()
df[['Country', 'City']] = df['IP Address'].progress_apply(get_location)

# Đóng file
reader.close()

100%|██████████| 94080/94080 [00:38<00:00, 2473.16it/s]


In [236]:
df['City'].value_counts().head(10)

City
Hanoi               18891
Ho Chi Minh City    15873
Da Nang              2128
Biên Hòa             1265
Singapore            1100
Chicago              1046
Nha Trang             925
Quận Bình Thạnh       836
Can Tho               806
Ha Long               664
Name: count, dtype: int64

In [237]:
df['City'] = df['City'].apply(lambda x: x if pd.isnull(x) or 'Quận' not in x else 'Can Tho' if 'Ninh Kiều' in x else 'Ho Chi Minh City')

In [238]:
df[['City','Country']].value_counts().head(10)

City              Country      
Ho Chi Minh City  Vietnam          19289
Hanoi             Vietnam          18891
Da Nang           Vietnam           2128
Biên Hòa          Vietnam           1265
Singapore         Singapore         1100
Chicago           United States     1046
Can Tho           Vietnam            948
Nha Trang         Vietnam            925
Ha Long           Vietnam            664
Haiphong          Vietnam            662
Name: count, dtype: int64

<h2>Phân loại chủ đề tìm kiếm.

In [239]:
# Tạo dictionary cho các từ khóa
theme = {'Phân tích dữ liệu': ['phân tích dữ liệu', 'data','analytics','dữ liệu'],
         'Young Talent':['young talent'],
         'Excel':['excel'],
         'Sql':['sql'],
         'Power Bi':['power bi','powerbi'],
         'Áp lực cuộc sống':['áp lực cuộc sống'],
         'Tư duy': ['tư duy'],
         'CAREER COACH': ['career coach']}

In [242]:
# Hàm phân loại tên theo từ khóa
def phan_loai(name, vocab):
    if pd.isna(name):
        return None
    else:
        matched = False
        for key, value in vocab.items():
            for v in value:
                if v in name.lower().replace('-',' '):
                    matched = True
                    return key
                    break
            if matched:
                break
        if not matched:
            return 'Khác'

In [243]:
df['Category'] = df['Title'].apply(lambda x: phan_loai(x, theme))

In [244]:
df['Category'].value_counts()

Category
Khác                 27919
Phân tích dữ liệu    25611
Young Talent         14612
Excel                 9252
Tư duy                1617
Power Bi               619
Áp lực cuộc sống       599
CAREER COACH           462
Sql                    173
Name: count, dtype: int64

In [245]:
df.head(3)

,Email,Type,Name,Title,MA URL,MA Referrer,ma_path,IP Address,cuid,Date,Message Id,Template Id,Tag,Refer,Main_Domain,Country,City,Category
0,NaN,page,Cách lấy dữ liệu từ bảng này sang bảng khác tr...,Cách lấy dữ liệu từ bảng này sang bảng khác tr...,https://uniace.vn/cach-lay-du-lieu-tu-bang-nay...,https://www.google.com/,/cach-lay-du-lieu-tu-bang-nay-sang-bang-khac-t...,14.190.23.197,b2e2a96e-5b97-4880-abeb-35ebf7635b0c,1/8/2021 0:00,NaN,NaN,NaN,google.com,google,Vietnam,Hải Dương,Phân tích dữ liệu
1,NaN,page,Unique Analytics Center for Everyone - uniace.vn,Unique Analytics Center for Everyone - uniace.vn,https://uniace.vn/,https://www.google.com/,/,113.173.234.231,1d6772fe-ed2e-45a0-a5a1-d9a968cabe0b,1/8/2021 0:08,NaN,NaN,NaN,google.com,google,Vietnam,Ho Chi Minh City,Phân tích dữ liệu
2,NaN,page,Cách Sử Dụng Hàm Lọc Dữ Liệu Trùng Nhau Trong ...,Cách Sử Dụng Hàm Lọc Dữ Liệu Trùng Nhau Trong ...,https://uniace.vn/ham-loc-du-lieu-trung-nhau-t...,https://www.google.com/,/ham-loc-du-lieu-trung-nhau-trong-excel/,171.252.153.188,fc75c68e-8fba-44b0-8723-b38e1af30cc0,1/8/2021 0:08,NaN,NaN,NaN,google.com,google,Vietnam,Ho Chi Minh City,Phân tích dữ liệu


<h2>Phân loại người dùng dựa trên Email.

In [246]:
df['Email'].value_counts()

Email
kieuanh4003@gmail.com          1368
hoangyen4394@gmail.com         1200
vothingocthao2001@gmail.com    1023
hieubh19@uef.edu.vn             753
nm.hoang19@gmail.com            689
                               ... 
uniace@outlook.com                1
u12@outlook.com                   1
trinhle3838@gmail.com             1
u13@outlook.com                   1
angle3652000@gmail.com            1
Name: count, Length: 2437, dtype: int64

In [247]:
# Tạo cột phân loại người dùng theo email
df['Customer_type'] = df['Email'].apply(lambda x: 'Normal' if pd.isna(x) else 'Office Staff' if 'outlook' in x else 'Student' if 'edu' in x else 'Normal')

In [248]:
df.Customer_type.value_counts()

Customer_type
Normal          89512
Student          2571
Office Staff     1997
Name: count, dtype: int64

In [249]:
# Tạo cột tên trường nếu người dùng là sinh viên
df['Uni'] = df['Email'].str.extract(r'@(?:[\w]+\.)?(.+)\.edu')

In [250]:
df.Uni.value_counts().head()

Uni
uef    795
ueh    636
ctu    342
uel    340
buh     72
Name: count, dtype: int64

<h2>Tạo cột đếm lượt khách hàng chỉ click 1 lần và không làm gì thêm (Bounce Rate).

In [251]:
# Tổng số truy cập không có email, ip address và cuid
no_in4 = df[pd.isnull(df['Email']) & (pd.isnull(df['IP Address'])) & (pd.isnull(df['cuid']))]
print('Tổng số truy cập không có email, ip address và cuid: ',len(no_in4))

Tổng số truy cập không có email, ip address và cuid:  4697


In [252]:
no_in4.index

Index([15432, 15433, 15434, 15435, 15436, 15437, 15438, 15439, 15440, 15441,
       ...
       20506, 20507, 20508, 20509, 25600, 32110, 32111, 32113, 32114, 32119],
      dtype='int64', length=4697)

In [253]:
# Tạo hàm lọc các bản ghi có email, ip address và cuid chỉ xuất hiện 1 lần
def once(col):
    filt_chart = df[df[col].isin(df[col].value_counts()[df[col].value_counts() == 1].index)]
    return filt_chart[['Email','IP Address','cuid']]

In [254]:
mail = once('Email')
ip = once('IP Address')
cuid = once('cuid')

In [255]:
bounce = pd.concat([mail,ip,cuid]).drop_duplicates()

In [256]:
bounce1 = pd.concat([mail,ip,cuid]).drop_duplicates()

In [257]:
len(bounce1)

7256

In [258]:
bounce.head()

,Email,IP Address,cuid
404,dolam98@gmail.com,NaN,NaN
9901,angle3652000@gmail.com,NaN,NaN
10306,dangdinhtran0@gmail.com,NaN,NaN
10906,anhlai721997@gmail.com,NaN,ddaf5d13-1395-4053-a51b-07b6470de902
33078,vpta8902@gmail.com,NaN,NaN


In [ ]:
# Tạo cột Bounce trong DataFrame df và đánh dấu các bản ghi có trong danh sách bounce
list_bounce = list(bounce.index) + list(no_in4.index) # Kết hợp các chỉ số từ bounce và no_in4
df['Bounce'] = 0 # Tạo cột Bounce với giá trị mặc định là 0
for i in list_bounce:
    df.iloc[i,20] = 1

In [ ]:
'''Ghép các bản ghi có email, ip address và cuid chỉ xuất hiện 1 lần sau khi đã loại bỏ c
 các bản ghi trùng lặp và đếm số lượng'''

once_time = len(bounce)
print('Tổng số bản ghi có email, ip address và cuid chỉ xuất hiện 1 lần: ',once_time)

Tổng số bản ghi có email, ip address và cuid chỉ xuất hiện 1 lần:  7256


In [ ]:
''' Tính Bounce Rate bằng cách lấy tổng số truy cập không có email, ip address và cuid(no_in4) + tổng số bản ghi
có email, ip address và cuid chỉ xuất hiện 1 lần(once_time) chia cho tổng số bản ghi trong DataFrame(df) '''

bounce_rate = round((len(no_in4) + once_time) / len(df) * 100,2)
print('Bounce Rate: ',bounce_rate,'%')

Bounce Rate:  12.71 %


In [ ]:
add_rate = round(len(df[df['Type'] == 'list_addition']) / len(df) * 100,2)
print('Add to card rate: ',add_rate,'%')

Add to card rate:  7.1 %


In [ ]:
len(df[df['Type'] == 'list_addition'])

6676

In [ ]:
conversion_rate = round(len(df[df['Tag'] == '["Completed Order"]']) / len(df) * 100,2)
print('Conversion Rate: ',conversion_rate,'%')

Conversion Rate:  0.94 %


<h2>Lưu file đã làm sạch

In [211]:
df.to_csv('Uniace_cleaned.csv',index=False)